# Experimentos

In [32]:
import time
import pandas as pd
from lcg import LCG
from xorshift import Xorshift64
from primality_tests import fermat_primality_test, miller_rabin

In [33]:
def generate(generator, num_bits):
    """
    Usa um gerador (LCG ou Xorshift) para construir um número pseudo-aleatório
    com um tamanho específico de bits.
    """
    if num_bits == 0:
        return 0

    large_number = 0
    created_bits = 0
    bits_per_call = generator.m.bit_length() - 1 if hasattr(generator, 'm') else 64

    # Gera o número pseudo-aleatório
    while created_bits < num_bits:
        random_number = generator.next()
        large_number = (large_number << bits_per_call) + random_number
        created_bits += bits_per_call
    
    excess_bits = created_bits - num_bits
    
    # Garante que o número gerado tenha o tamanho correto
    if excess_bits > 0:
        large_number = large_number >> excess_bits
    
    if num_bits > 0:
        # Garante o bit mais significativo tenha o comprimento correto.
        # (1 << (num_bits - 1)) cria um número com apenas o n-ésimo bit ligado.
        # A operação OU (|) liga esse bit, garantindo o tamanho.
        large_number |= (1 << (num_bits - 1))
        
        # Altera o bit menos significativo para garantir que o número seja ímpar.
        large_number |= 1
        
    return large_number

## Linear Congruent Generator (LCG)

In [34]:
print("--- Experimento com LCG ---")
seed_lcg = int(time.time())
lcg_gen = LCG(seed=seed_lcg, multiplier=1103515245, increment=12345, modulus=2**31)

size_bits = 256
print(f"Gerando número de {size_bits} bits com LCG (Seed: {seed_lcg})")

start_time = time.perf_counter()
final_number_lcg = generate(lcg_gen, size_bits)
end_time = time.perf_counter()

time_ms_lcg = (end_time - start_time) * 1000

print(f"\nTempo de execução: {time_ms_lcg:.4f} ms")
print(f"Número gerado: {final_number_lcg}") 
print(f"\nNumero de bits real: {final_number_lcg.bit_length()}")

--- Experimento com LCG ---
Gerando número de 256 bits com LCG (Seed: 1759189509)

Tempo de execução: 0.0448 ms
Número gerado: 68828553762044946295207278973791987467195977602000947349791384284349998611737

Numero de bits real: 256


## Xorshift

In [35]:
print("\n--- Experimento com Xorshift ---")
seed_xs = int(time.time())
xorshift_gen = Xorshift64(seed=seed_xs)

size_bits = 256
print(f"Gerando número de {size_bits} bits com Xorshift (Seed: {seed_xs})")

start_time = time.perf_counter()
final_number_xs = generate(xorshift_gen, size_bits)
end_time = time.perf_counter()

time_ms_xs = (end_time - start_time) * 1000

print(f"\nTempo de execução: {time_ms_xs:.4f} ms")
print(f"Número gerado: {final_number_xs}") 
print(f"\nNumero de bits real: {final_number_xs.bit_length()}")


--- Experimento com Xorshift ---
Gerando número de 256 bits com Xorshift (Seed: 1759189509)

Tempo de execução: 0.0544 ms
Número gerado: 69660527433212879918690718774540373077497181736134586758915026658593443172115

Numero de bits real: 256


## Geração da Tabela

In [ ]:
tamanhos_bits = [40, 56, 80, 128, 168, 224, 256, 512, 1024, 2048, 4096]
resultados = [] # Lista para guardar os resultados de cada execução

print("--- Iniciando o Experimento de Geração de Números ---")

for size in tamanhos_bits:
    print(f"Randomizando: {size} bits...")
    
    # Teste para LCG
    seed_lcg = int(time.time())
    lcg_gen = LCG(seed=seed_lcg, multiplier=1103515245, increment=12345, modulus=2**31)
    
    start_time_lcg = time.perf_counter()
    generate(lcg_gen, size)
    end_time_lcg = time.perf_counter()
    time_ms_lcg = (end_time_lcg - start_time_lcg) * 1000
    
    resultados.append({"Algoritmo": "LCG", "Tamanho (bits)": size, "Tempo (ms)": time_ms_lcg})
    
    # Teste para Xorshift
    seed_xs = int(time.time())
    xorshift_gen = Xorshift64(seed=seed_xs)
    
    start_time_xs = time.perf_counter()
    generate(xorshift_gen, size)
    end_time_xs = time.perf_counter()
    time_ms_xs = (end_time_xs - start_time_xs) * 1000
    
    resultados.append({"Algoritmo": "Xorshift", "Tamanho (bits)": size, "Tempo (ms)": time_ms_xs})

print("\n--- Experimento Concluído ---")

# Converte a lista de resultados para um DataFrame do pandas facilitando a visualização
df_resultados = pd.DataFrame(resultados)

# Exibe a tabela de resultados
print("\n Tabela de Desempenho dos Geradores")
print("-" * 40)
print(df_resultados.to_string())

--- Iniciando o Experimento de Geração de Números ---
Randomizando: 40 bits...
Randomizando: 56 bits...
Randomizando: 80 bits...
Randomizando: 128 bits...
Randomizando: 168 bits...
Randomizando: 224 bits...
Randomizando: 256 bits...
Randomizando: 512 bits...
Randomizando: 1024 bits...
Randomizando: 2048 bits...
Randomizando: 4096 bits...

--- Experimento Concluído ---

 Tabela de Desempenho dos Geradores
----------------------------------------
   Algoritmo  Tamanho (bits)  Tempo (ms)
0        LCG              40      0.0085
1   Xorshift              40      0.0033
2        LCG              56      0.0032
3   Xorshift              56      0.0022
4        LCG              80      0.0028
5   Xorshift              80      0.0049
6        LCG             128      0.0038
7   Xorshift             128      0.0021
8        LCG             168      0.0035
9   Xorshift             168      0.0030
10       LCG             224      0.0055
11  Xorshift             224      0.0036
12       LCG      

# Testes de Primalidade

### Miller Rabin (MR)

In [37]:
bit_size = [40, 56, 80, 128, 168, 224, 256, 512, 1024, 2048, 4096]
results_mr = []

print("--- Iniciando Busca de Primos com Miller-Rabin ---")

for size in bit_size:
    print(f"Buscando um primo de {size} bits...")
    generator_mr = Xorshift64(seed=int(time.time()))
    start_time = time.perf_counter()
    
    while True:
        candidate = generate(generator_mr, size)
        if miller_rabin(candidate):
            prime_found = candidate
            break # Primo foi encontrado
            
    end_time = time.perf_counter()
    time_ms = (end_time - start_time) * 1000
    
    # Adiciona o resultado
    results_mr.append({
        "Tamanho (bits)": size,
        "Tempo para encontrar (ms)": time_ms,
        "Primo Encontrado": prime_found
    })

print("\n--- Concluído (Miller-Rabin) ---")

# Apresenta a tabela
df_mr = pd.DataFrame(results_mr)

# Configuração para garantir que os números grandes sejam exibidos por completo
pd.set_option('display.max_colwidth', None)

print("\n Tabela de Resultados: Busca com Miller-Rabin")
print("-" * 40)
print(df_mr.to_string())

--- Iniciando Busca de Primos com Miller-Rabin ---
Buscando um primo de 40 bits...
Buscando um primo de 56 bits...
Buscando um primo de 80 bits...
Buscando um primo de 128 bits...
Buscando um primo de 168 bits...
Buscando um primo de 224 bits...
Buscando um primo de 256 bits...
Buscando um primo de 512 bits...
Buscando um primo de 1024 bits...
Buscando um primo de 2048 bits...
Buscando um primo de 4096 bits...

--- Concluído (Miller-Rabin) ---

 Tabela de Resultados: Busca com Miller-Rabin
----------------------------------------
    Tamanho (bits)  Tempo para encontrar (ms)                                                                                                                                                                                                                                                                                                                                                                                                                                   

### Fermat

In [38]:
bit_size = [40, 56, 80, 128, 168, 224, 256, 512, 1024, 2048, 4096]
results_ft = []

print("--- Iniciando Busca de Primos com Teste de Fermat ---")

for size in bit_size:
    print(f"Buscando um primo de {size} bits...")
    generator_ft = Xorshift64(seed=int(time.time()))
    start_time = time.perf_counter()
    
    while True:
        candidate = generate(generator_ft, size)
        if fermat_primality_test(candidate):
            prime_found = candidate
            break # Primo foi encontrado
            
    end_time = time.perf_counter()
    time_ms = (end_time - start_time) * 1000
    
    # Adiciona o resultado
    results_ft.append({
        "Tamanho (bits)": size,
        "Tempo para encontrar (ms)": time_ms,
        "Primo Encontrado": prime_found
    })

print("\n--- Concluído (Fermat) ---")

# Apresenta a tabela
df_ft = pd.DataFrame(results_ft)

# Configuração para garantir que os números grandes sejam exibidos por completo
pd.set_option('display.max_colwidth', None)

print("\n Tabela de Resultados: Busca com Teste de Fermat")
print("-" * 40)
print(df_ft.to_string())

--- Iniciando Busca de Primos com Teste de Fermat ---
Buscando um primo de 40 bits...
Buscando um primo de 56 bits...
Buscando um primo de 80 bits...
Buscando um primo de 128 bits...
Buscando um primo de 168 bits...
Buscando um primo de 224 bits...
Buscando um primo de 256 bits...
Buscando um primo de 512 bits...
Buscando um primo de 1024 bits...
Buscando um primo de 2048 bits...
Buscando um primo de 4096 bits...

--- Concluído (Fermat) ---

 Tabela de Resultados: Busca com Teste de Fermat
----------------------------------------
    Tamanho (bits)  Tempo para encontrar (ms)                                                                                                                                                                                                                                                                                                                                                                                                                                   